In [1]:
import os
import math
import quandl
import itertools
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
KNeighboursClassifier = KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
%matplotlib inline
# Your code goes here

C:\ProgramData\Anaconda3\envs\cryptoalgo\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import requests

In [3]:
import json

In [5]:
#quandl database query through API
quandl.ApiConfig.api_key = 'gTmSNjbQR-8Q5U9pukHX'
quandl.get('BITFINEX/BTCEUR',collapse="monthly")


High          Low      Mid    Last     Bid     Ask       Volume
Date                                                                         
2017-11-30  9647.3  7596.798998  8243.85  8268.6  8241.0  8246.7  8856.520924
2017-12-31  9768.0  9033.100000  9768.25  9768.0  9767.8  9768.7  2370.582143

In [6]:
#downloaded dataset for daily price
data_path = os.path.join(os.getcwd(), 'datasets', 'cryptocurrencyhistoricaldata','ethereum.csv')
train_A = pd.read_csv(data_path, delimiter = ';')

In [7]:
#general coin informaiton from crypto compare
response = requests.get("https://www.cryptocompare.com/api/data/coinlist/")
coinInfo = response.json()
coinInfo = coinInfo['Data']
json.dumps(coinInfo)
coinInfo = pd.read_json(json.dumps(coinInfo),orient='index')

In [8]:
#retriving (eth) price from crypto compare
parameters = {"fsym":"ETH","tsym":"BTC"}
response = requests.get("https://min-api.cryptocompare.com/data/histominute",params=parameters)
ethPriceByMin = response.json()
ethPriceByMin = ethPriceByMin["Data"]
ethPriceByMin = pd.read_json(json.dumps(ethPriceByMin),orient="columns")


In [10]:
# retriveing from GDAX using unofficial client package
#importing GDAX package from local directory
import sys

sys.path.insert(0,'C:/Users/Yong Zhen Yu/Documents/gdax-python/gdax')
sys.path.insert(0,'C:/Users/Yong Zhen Yu/Documents/gdax-python')

import gdax

public_client = gdax.PublicClient()

len(public_client.get_product_historic_rates("ETH-USD"))

401

In [34]:
# retriveing from Kraken using unofficial client package Krakenex
# attempting to retrive directly from github
#kraken API seems to be rather unstable


import krakenex

k = krakenex.API()

#gets the price pair "keys to use below
k.query_public("AssetPairs")["result"].keys()

parameters = {"pair":"XETHZEUR"}

#draws live order book
k.query_public("Depth",parameters)

{'error': [],
 'result': {'XETHZEUR': {'asks': [['380.96000', '165.083', 1512571884],
    ['380.97000', '200.000', 1512571131],
    ['380.98000', '200.000', 1512571500],
    ['380.99000', '200.000', 1512571585],
    ['381.00000', '209.950', 1512571850],
    ['381.02000', '0.680', 1512570741],
    ['381.05000', '200.680', 1512570893],
    ['381.10000', '400.000', 1512570690],
    ['381.20000', '400.000', 1512571046],
    ['381.49000', '0.680', 1512571839],
    ['381.50000', '3.286', 1512571418],
    ['381.58000', '204.549', 1512571808],
    ['381.65000', '10.300', 1512571336],
    ['381.71000', '1.361', 1512571882],
    ['381.76000', '5.000', 1512570926],
    ['381.77000', '30.933', 1512570234],
    ['381.78000', '0.056', 1512569711],
    ['381.81000', '0.681', 1512571871],
    ['381.90000', '0.200', 1512562844],
    ['381.96000', '0.335', 1512565253],
    ['382.00000', '1080.498', 1512570401],
    ['382.02000', '0.336', 1512564055],
    ['382.05000', '25.000', 1512571864],
    ['382.06

In [4]:
#retriveing data form bitcoinity.org

response = requests.get("https://data.bitcoinity.org/markets/bidask_sum/24h/USD/bitfinex?bp=1&bu=c&r=minute&t=m")

In [9]:
from bs4 import BeautifulSoup

In [10]:
bitoinityPage = BeautifulSoup(response.content,'html.parser')

In [15]:
chartParams = {"data_type":"bidask_sum","currency":"USD","exchange":"bitfinex","compare":"no","bidask_percent":"1","bidask_unit":"curr","function":"none","timespan":"24h","groups_count":10,"resolution":"minute","chart_type":"multi","smoothing":"linear","scale_type":"lin"}

In [19]:
#post request to chart_data with chartParams, gets back data
response = requests.post("https://data.bitcoinity.org/chart_data", data=chartParams)
response.content

b'{"data":[{"key":"asks","values":[[1512495540000,4446268.24543722],[1512495600000,4434707.79704202],[1512495660000,4841208.80538009],[1512495720000,5113372.4262299],[1512495780000,4848692.93524843],[1512495840000,5309319.50584695],[1512495900000,5796214.95535997],[1512495960000,5669034.91071457],[1512496020000,5694735.96355588],[1512496080000,6003544.77308402],[1512496140000,5985786.60273374],[1512496200000,5650506.04453919],[1512496260000,5507841.42926839],[1512496320000,8254610.75554941],[1512496380000,4509563.18444628],[1512496440000,4301165.84765586],[1512496500000,4231532.05996424],[1512496560000,3834271.79309995],[1512496620000,3425068.18528938],[1512496680000,2778221.75498164],[1512496740000,2788054.23520452],[1512496800000,2520348.5439776],[1512496860000,2678853.006586],[1512496920000,2619323.65408175],[1512496980000,3222835.20256164],[1512497040000,4358104.88055719],[1512497100000,4679954.87185537],[1512497160000,4581022.71543808],[1512497220000,3968074.09485881],[15124972800

In [14]:
req = requests.Request?


In [11]:
print(bitoinityPage.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Bitcoinity.org
  </title>
  <link href="//static.bitcoinity.org/assets/data-screen-b711e36db928164740bfd2e46e697736.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="//static.bitcoinity.org/assets/data-print-ed1a301e961af17ff5b75c2fd441fa10.css" media="print" rel="stylesheet" type="text/css"/>
  <script src="//static.bitcoinity.org/assets/data-1cf3554a12fc117a95b8da73b5fda480.js" type="text/javascript">
  </script>
  <link href="/data_favicon.ico" rel="icon" type="image/x-icon"/>
  <script src="//static.bitcoinity.org/assets/c3-175a720d1950508bb718795d2d0d8e16.js" type="text/javascript">
  </script>
  <link href="//static.bitcoinity.org/assets/c3-e33e3766d485785cb6ba50a4993fb0a3.css" media="screen" rel="stylesheet" type="text/css"/>
  <meta content="authenticity_token" name="csrf-param"/>
  <meta content="ogl4qX9e5Qp7fme+xujScdZlE0/s3HlkAoH+KPUVb/I=" name="csrf-token"/>
 </head>
 <body>
  <div class="main_title row-fluid">


In [4]:
from bitcoinity import BitcoinityScraper

In [5]:
bts = BitcoinityScraper()

In [6]:
bts.get_chart_data()

b'{"data":[{"key":"asks","values":[[1512576840000,10819236.8580624],[1512576900000,11879886.2826334],[1512576960000,13145520.5160501],[1512577020000,10357150.7183197],[1512577080000,10438648.4563853],[1512577140000,9645484.16027214],[1512577200000,8685258.71778965],[1512577260000,8386082.05984009],[1512577320000,7830605.53267719],[1512577380000,8282935.18306717],[1512577440000,8357494.26992621],[1512577500000,8403947.17689828],[1512577560000,8697409.66583151],[1512577620000,8476929.11817364],[1512577680000,8393958.99233397],[1512577740000,9436234.38704597],[1512577800000,9289371.14507913],[1512577860000,9239501.72749586],[1512577920000,9166391.65763416],[1512577980000,6328511.36476679],[1512578040000,6111233.52905012],[1512578100000,8856011.08195945],[1512578160000,8915926.69908419],[1512578280000,7911433.34275212],[1512578340000,8904981.8174204],[1512578400000,8841857.00022322],[1512578460000,9351549.93657077],[1512578520000,8388520.61002333],[1512578580000,10372102.3936031],[15125786